### Importing the necessary libraries

In [42]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

##### Step 1:. There are several CSV files for each month.
##### Step 2: Merging all the CSV files into one DataFrame.
##### Step 3: Exploratory data analysis.
##### Step 4: Answering Questions.

In [43]:
# Creating a path
folder_path = 'C:/Users/ladyb/Desktop/My Portfolio/Ya Sals/Sales_Data/'

# Getting the files
csvfiles = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Creating an empty dataframe
merged_data = pd.DataFrame()

# Looping CSV and appending
for file in csvfiles:
    file_path = os.path.join(folder_path, file)
    data = pd.read_csv(file_path)
    merged_data = merged_data.append(data, ignore_index=True)

In [44]:
df = merged_data

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [46]:
# Checking the presence of null values
display(df.isnull().sum())

# Removing null values
df = df.dropna()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [47]:
# The DataFrame couldn't convert the column to float because it has a value written 'Price Each'
# Hence the code below is to locate these values in whole of DataFrame
display(df[df['Price Each'].astype(str) == 'Price Each'])

# Removing these unwanted values
df = df[df['Price Each'].astype(str) != 'Price Each']

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
519,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1149,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
1155,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2878,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
2893,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
...,...,...,...,...,...,...
185164,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
185551,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186563,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
186632,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address


In [51]:
# Conversion of the columns with numerical values
df['Price Each'] = df['Price Each'].astype(float)
df['Quantity Ordered'] = df['Quantity Ordered'].astype(int)

# Answering Questions

#### 1. What was the best month for sales? How much was earned that month?

In [53]:
# Extracting first two numbers on column Order date which represents Months

df['Months'] = df['Order Date'].str[:2]

In [60]:
display(df.groupby('Months')[['Price Each']].sum().sort_values(by ='Price Each', ascending=False))

print('The best month was December and 4588415.41 was earned.')

,Price Each
Months,
12,4588415.41
10,3715554.83
04,3367671.02
11,3180600.68
05,3135125.13
03,2791207.83
07,2632539.56
06,2562025.61
08,2230345.42


The best month was December and 4588415.41 was earned.


#### 2. What city sold the most product?

In [67]:
# Extracting cities from the Purchase Address column

def extract_city(Address):
    return Address.split(',')[1]

df['city'] = df['Purchase Address'].apply(extract_city)

In [71]:
display(df.groupby('city')[['Price Each']].sum().sort_values(by='Price Each', ascending=False))

print('The city that sold the most products is San Francisco')

,Price Each
city,
San Francisco,8211461.74
Los Angeles,5421435.23
New York City,4635370.83
Boston,3637409.77
Atlanta,2779908.20
Dallas,2752627.82
Seattle,2733296.01
Portland,2307747.47
Austin,1809873.61


The city that sold the most products is San Francisco


#### 3. What time should we display advertisements to maximize the likelihood of customer’s buying product?

In [75]:
# 1. Extracting time from the order date. Extract only hours 
# 2. Assuming the time with the highest sales is the time to display ads to maximize customer’s buying of product
df['Hour'] = df['Order Date'].str[8:-3]

In [81]:
display(df.groupby('Hour')[['Price Each']].sum().sort_values(by='Price Each', ascending=False))

print('Around 1900 hours is the time to display ads and maximize the likelihood of sales since around this time the sales may have been contributed by the fact that its the time most people are coming off work and purchase things to go home with.')

,Price Each
Hour,
19,2398588.31
12,2299876.68
11,2288855.18
20,2268185.16
18,2207696.93
13,2139743.86
17,2116777.02
14,2072194.77
21,2030763.83


Around 1900 hours is the time to display ads and maximize the likelihood of sales since around this time the sales may have been contributed by the fact that its the time most people are coming off work and purchase things to go home with.


#### 4. What products are most often sold together?

In [94]:
from itertools import combinations

In [97]:
# create a dictionary to store the frequency of each itemset
itemsets = {}

# loop through each transaction
for trans_id, group in df.groupby('Order ID'):
    # get a list of the products in the transaction
    products = list(group['Product'])
    # loop through each combination of products in the transaction
    for i in range(1, len(products)+1):
        for combo in combinations(products, i):
            # sort the items in the itemset to remove duplicates
            itemset = tuple(sorted(combo))
            # increment the count for the itemset
            itemsets[itemset] = itemsets.get(itemset, 0) + 1

# create a dataframe to store the frequency of each itemset
freq_df = pd.DataFrame.from_dict(itemsets, orient='index', columns=['Frequency'])

# add a column for the size of each itemset
freq_df['Size'] = freq_df.index.map(len)

# filter the dataframe to show only itemsets with at least two items
freq_df = freq_df[freq_df['Size'] > 1]

# sort the dataframe by frequency in descending order
freq_df = freq_df.sort_values('Frequency', ascending=False)

# print the top 10 frequent itemsets
print(freq_df.head(10))

print('>>> Items that are mostly sold together are Lightning Charging Cable, iPhone')

                                              Frequency  Size
(Lightning Charging Cable, iPhone)                 1015     2
(Google Phone, USB-C Charging Cable)                999     2
(Wired Headphones, iPhone)                          462     2
(Google Phone, Wired Headphones)                    423     2
(Apple Airpods Headphones, iPhone)                  373     2
(USB-C Charging Cable, Vareebadd Phone)             368     2
(Bose SoundSport Headphones, Google Phone)          228     2
(USB-C Charging Cable, Wired Headphones)            205     2
(Vareebadd Phone, Wired Headphones)                 149     2
(Lightning Charging Cable, Wired Headphones)        129     2
>>> Items that are mostly sold together are Lightning Charging Cable, iPhone


#### 5. What product sold the most? Why do you think it sold the most?

In [110]:
display(df.groupby('Product')[['Price Each']].sum().sort_values(by='Price Each', ascending=False))

print('Product that sold most is Macbook Pro Laptop.')
print('The reasons it sold more is because of brand reputation, features and Market demand.')

,Price Each
Product,
Macbook Pro Laptop,8030800.00
iPhone,4789400.00
ThinkPad Laptop,4127958.72
Google Phone,3315000.00
27in 4K Gaming Monitor,2429637.70
34in Ultrawide Monitor,2348718.19
Apple Airpods Headphones,2332350.00
Flatscreen TV,1440000.00
Bose SoundSport Headphones,1332366.75


Product that sold most is Macbook Pro Laptop.
The reasons it sold more is because of brand reputation, features and Market demand.
